# Restaurant Review Sentiment Analysis usind DeepLearning

This notebook performs sentiment analysis on restaurant reviews using a neural network model. It classifies reviews as positive or negative based on the sentiment expressed in the text.

In [22]:
#restaurant_review_sentiment_analysis.ipynb
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#@title Restaurant Review Sentiment Analysis

# ---------------------- Import Libraries --------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from keras.models import Sequential
from keras.layers import Dense




# ------------------- Load and Preprocess Data --------------------------------

# Load the review data from .tsv file
dataset = pd.read_csv('/content/dataset.tsv', delimiter='\t', quoting=3)

# Data cleaning and preprocessing
def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    text = [ps.stem(word) for word in text if word not in stop_words]
    text = ' '.join(text)
    return text

dataset['Review'] = dataset['Review'].apply(preprocess_text)




# --------------------- Split the Data -----------------------------------------

# Split the data into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(dataset['Review'], dataset['Liked'], test_size=0.2, random_state=42)



# ---------------------- Vectorization -----------------------------------------

# Create a CountVectorizer object for vectorization
vectorizer = CountVectorizer()

# Fit and transform the train data into a Bag-of-Words matrix
train_matrix = vectorizer.fit_transform(train_data).toarray()

# Transform the test data into a Bag-of-Words matrix
test_matrix = vectorizer.transform(test_data).toarray()




# ------------------ Build the Neural Network Model ----------------------------

# Define the neural network architecture
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=train_matrix.shape[1]))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




# ----------------------- Train the Model --------------------------------------

# Train the model and store the training history
history = model.fit(train_matrix, train_labels, epochs=10, batch_size=32)




# --------------------- Evaluate the Model ------------------------------------

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_matrix, test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)




# ---------------------- Make Predictions --------------------------------------

# Make predictions on the test data
test_predictions = model.predict(test_matrix)
test_predictions = (test_predictions > 0.5).astype(int)




# ------------------ Print Classification Report -------------------------------

# Print classification report
report = classification_report(test_labels, test_predictions)
print("Classification Report:\n", report)




# ------------------------ Plot Training Loss ----------------------------------

# Plot the training loss
plt.plot(history.history['loss'])
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()


In [15]:
#@title Save the model 
model.save("trained_model.pkl")

In [ ]:
!pip install gradio

In [17]:
#@title Interface
import gradio as gr
def predict_sentiment(review):
    review_matrix = vectorizer.transform([review]).toarray()
    prediction = model.predict(review_matrix)[0][0]
    sentiment = 'Positive' if prediction >= 0.5 else 'Negative'
    return sentiment
interface = gr.Interface(fn=predict_sentiment, inputs="text", outputs="text")
interface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>